# Regression model

- Want to predict relative performance

Goal: build a
We want to pursue opportunities where the benifits outwigh the risks


# Get raw data

In [4]:
import pandas as pd

data = pd.read_csv("../.data/interim/data_with_target.csv", index_col=0)

In [ ]:
# define target
data["Target"] = data.groupby('Ticker')['Close'].pct_change(1).shift(-3) # target becomes 1-day returns

# Feature engineering

In [7]:
import talib

def create_features(data):
    # ===== Macro features =====
    #data['VIX_ret'] = data['VIX'].pct_change() # very noisy fro day-to-day
    #data['WTI_ret'] = data['WTI_Oil'].pct_change()
    #data['TNX_ret'] = data['US10Y'].pct_change()
    data['VIX_5d_mean'] = data['VIX'].rolling(5).mean()
    data['WTI_5d_std'] = data['WTI_Oil'].rolling(5).std()

    # ===== Market indices =====
    market_indices = ["GSPC", "NDX", "RUT", "DJI"]
    for idx in market_indices:
        data[f'{idx}_return_1d'] = data[idx].pct_change()
        data[f'{idx}_return_5d'] = data[idx].pct_change(5)

    # ===== Stock returns =====
    for period in [1, 3, 5, 10]:
        data[f'return_{period}d'] = data.groupby('Ticker')['Close'].pct_change(period)

    # ===== Rolling volatility =====
    data['rolling_std_5'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(5).std())
    data['rolling_std_10'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(10).std())


    # ===== Moving averages and ratios =====
    data['ma_5'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(5).mean())
    data['ma_20'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).mean())
    data['price_ma5_ratio'] = data['Close'] / data['ma_5']
    data['price_ma20_ratio'] = data['Close'] / data['ma_20']
    data['MA5_MA20_diff'] = data['ma_5'] - data['ma_20']
    data['MA5_MA20_ratio'] = data['ma_5'] / data['ma_20']

    # ===== Volume features =====
    data['vol_MA5'] = data.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(5).mean())
    data['vol_MA20'] = data.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(20).mean())
    data['vol_ratio_5'] = data['Volume'] / data['vol_MA5']
    data['vol_ratio_20'] = data['Volume'] / data['vol_MA20']
    # data['price_volume'] = data['return_1d'] * data['Volume']

    # ===== Momentum indicators =====
    data['rsi'] = data.groupby('Ticker')['Close'].transform(lambda x: talib.RSI(x.values, timeperiod=14))
    data['macd'] = data.groupby('Ticker')['Close'].transform(lambda x: x.ewm(span=12, adjust=False).mean() - x.ewm(span=26, adjust=False).mean())
    data['macd_signal'] = data.groupby('Ticker')['macd'].transform(lambda x: x.ewm(span=9, adjust=False).mean())
    data['macd_hist'] = data['macd'] - data['macd_signal']
    data.drop(columns=['macd_signal'], inplace=True)

    # ===== True Range & ATR =====
    data['tr'] = data['High'] - data['Low']
    data['tr1'] = abs(data['High'] - data['Close'].shift(1))
    data['tr2'] = abs(data['Low'] - data['Close'].shift(1))
    data['true_range'] = data[['tr', 'tr1', 'tr2']].max(axis=1)
    data['ATR_14'] = data.groupby('Ticker')['true_range'].transform(lambda x: x.rolling(14).mean())
    data['ATR_7'] = data.groupby('Ticker')['true_range'].transform(lambda x: x.rolling(7).mean())
    data.drop(columns=['tr','tr1','tr2','true_range'], inplace=True)

    # ===== Bollinger Bands =====
    ma20 = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).mean())
    std20 = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).std())
    data['BB_width'] = (2 * std20) / ma20

    # ===== Price Range Features =====
    data['range_pct'] = (data['High'] - data['Low']) / data['Close']
    data['OC_pct'] = (data['Close'] - data['Open']) / data['Close']
    data['range_5d'] = data.groupby('Ticker').apply(lambda x: (x['High'].rolling(5).max() - x['Low'].rolling(5).min()) / x['Close']).reset_index(level=0, drop=True)


    # handle NaNs
    data.bfill(inplace=True)
    data.ffill(inplace=True)

create_features(data)

C:\Users\maria\AppData\Local\Temp\ipykernel_25116\642010769.py:65: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data['range_5d'] = data.groupby('Ticker').apply(lambda x: (x['High'].rolling(5).max() - x['Low'].rolling(5).min()) / x['Close']).reset_index(level=0, drop=True)


### One-hot encore ticker values

In [8]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_values = encoder.fit_transform(data[['Ticker']])
new_cols = encoder.get_feature_names_out(['Ticker'])

tickers_encoded = pd.DataFrame(encoded_values, columns=new_cols, index=data.index)

data = pd.concat(
    [data, tickers_encoded],
    axis=1
)

In [10]:
data

,Date,Ticker,Close,High,Low,Open,Volume,WTI_Oil,US10Y,VIX,...,Ticker_KO,Ticker_MSFT,Ticker_NEE,Ticker_NVDA,Ticker_PFE,Ticker_QQQ,Ticker_SPY,Ticker_T,Ticker_VTI,Ticker_XOM
0,2020-01-02,AAPL,72.538528,72.598907,71.292319,71.545905,135480400,61.180000,1.882,12.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02,AMZN,94.900497,94.900497,93.207497,93.750000,80580000,61.180000,1.882,12.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-02,BA,331.348572,331.378393,325.761816,326.606765,4544400,61.180000,1.882,12.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-02,BAC,30.920332,30.937684,30.616682,30.668735,37614200,61.180000,1.882,12.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-02,BHP,33.608608,33.822793,33.522936,33.792196,1382193,61.180000,1.882,12.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25155,2024-12-31,QQQ,509.305664,515.711423,508.339314,514.954335,29117000,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25156,2024-12-31,SPY,580.989136,585.509524,579.343521,584.785824,57052700,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25157,2024-12-31,T,21.793722,21.870292,21.631011,21.650154,22342800,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
25158,2024-12-31,VTI,287.127228,289.435676,286.394088,289.108714,3893200,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### select features

In [11]:
returns = ['return_1d', 'return_3d', 'return_5d']
rolling_volatility = ['rolling_std_5', 'rolling_std_10']
moving_avg = ['ma_5', 'ma_20', 'price_ma5_ratio', 'price_ma20_ratio', 'MA5_MA20_diff'] # 'MA5_MA20_ratio'] 
momentum_indicators = [] # ['rsi', 'macd', 'macd_hist']
volume_features = [] #  ['price_volume'] # 'vol_MA5', 'vol_MA20', 'vol_ratio_5', 'vol_ratio_20', 
macro_features = [] # ['VIX_5d_mean', 'WTI_5d_std'] # 'VIX_ret', 'WTI_ret', 'TNX_ret',
market_index_features = [] # ["GSPC_return_1d", "NDX_return_1d", "RUT_return_1d", "DJI_return_1d", "GSPC_return_5d", "NDX_return_5d", "RUT_return_5d", "DJI_return_5d"] # (make almost no diff)
atr = ['ATR_14', 'ATR_7']
bb = ['BB_width']
price_range = [] # ['range_pct', 'OC_pct', 'range_5d']

feature_columns = returns\
    + rolling_volatility\
    + moving_avg\
    + momentum_indicators\
    + volume_features\
    + macro_features\
    + market_index_features\
    + atr\
    + bb\
    + price_range\
    + new_cols.tolist()


data[feature_columns] = data.groupby('Ticker')[feature_columns].shift(1) # prevent data leakage from using same-day data
data.dropna()

,Date,Ticker,Close,High,Low,Open,Volume,WTI_Oil,US10Y,VIX,...,Ticker_KO,Ticker_MSFT,Ticker_NEE,Ticker_NVDA,Ticker_PFE,Ticker_QQQ,Ticker_SPY,Ticker_T,Ticker_VTI,Ticker_XOM
40,2020-01-06,AAPL,72.405693,72.444336,70.703027,70.954203,118387200,63.270000,1.811,13.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,2020-01-06,AMZN,95.143997,95.184502,93.000000,93.000000,81236000,63.270000,1.811,13.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,2020-01-06,BA,331.766083,332.879454,325.940756,327.352341,5355000,63.270000,1.811,13.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,2020-01-06,BAC,30.234940,30.278321,29.818505,29.853208,42185000,63.270000,1.811,13.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,2020-01-06,BHP,33.345459,33.412776,33.272026,33.284266,771584,63.270000,1.811,13.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25155,2024-12-31,QQQ,509.305664,515.711423,508.339314,514.954335,29117000,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25156,2024-12-31,SPY,580.989136,585.509524,579.343521,584.785824,57052700,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
25157,2024-12-31,T,21.793722,21.870292,21.631011,21.650154,22342800,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
25158,2024-12-31,VTI,287.127258,289.435707,286.394118,289.108745,3893200,71.720001,4.573,17.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Make predicitons

In [12]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import joblib
from pathlib import Path

model = RandomForestRegressor(n_estimators=200, max_depth=8)

tscv = TimeSeriesSplit(n_splits=5, gap=3)  # gap=3 for your 3-day prediction

X = data[feature_columns].values
y = data['Target'].values
# Walk-forward validation
for train_idx, test_idx in tscv.split(X):
    model.fit(X[train_idx], y[train_idx])
    pred = model.predict(X[test_idx])
    
    # Evaluate as a trading strategy
    signals = np.sign(pred)  # 1 = long, -1 = short
    returns = signals * y[test_idx]
    
    print(f"Mean return: {returns.mean():.4f}")
    print(f"Sharpe: {returns.mean() / returns.std() * np.sqrt(252/3):.2f}")
    print(f"Hit rate: {(returns > 0).mean():.2%}")

Mean return: 0.0011
Sharpe: 0.60
Hit rate: 52.92%
Mean return: -0.0008
Sharpe: -0.34
Hit rate: 47.96%
Mean return: 0.0005
Sharpe: 0.23
Hit rate: 49.89%
Mean return: 0.0009
Sharpe: 0.51
Hit rate: 52.13%
Mean return: 0.0008
Sharpe: 0.49
Hit rate: 54.23%


In [13]:
model_path = Path("xgb_model.pkl")
joblib.dump(model, model_path)

['xgb_model.pkl']

# Backtest code
## Get new data for backtest

In [14]:
# get new data
import yfinance as yf

start_date = "2025-01-01"
end_date = "2025-10-12"

tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "JPM", "BAC", "JNJ", "PFE", "DIS", "KO", "BA", "XOM", "BHP", "NEE", "T", "SPY", "QQQ", "IWM", "VTI"]
macro_tickers = ['^VIX', 'CL=F', '^TNX']  # VIX, WTI Oil, 10Y Treasury
market_indices = ["^GSPC", "^NDX", "^RUT", "^DJI"]

# get stock data
stock_data = yf.download(tickers, start=start_date, end=end_date)
stock_data.stack(level=1).reset_index()

# get macro data
macro_data = yf.download(macro_tickers, start=start_date, end=end_date)['Close']
macro_data.rename(columns={'^VIX':'VIX', 'CL=F':'WTI_Oil', '^TNX':'US10Y'}, inplace=True)
macro_data = macro_data.ffill()

# get indices data
market_data = yf.download(market_indices, start=start_date, end=end_date)['Close']
market_data.rename(columns={'^GSPC':'GSPC', '^NDX':'NDX', '^RUT':'RUT', '^DJI':'DJI'}, inplace=True)

# putting tock data into long format
stock_data_long_format = stock_data.stack(level=1).reset_index()
stock_data_long_format = stock_data_long_format.merge(macro_data.reset_index(), on='Date', how='left')
stock_data_long_format = stock_data_long_format.merge(market_data.reset_index(), on='Date', how='left')

# calculate target: 3-day returns
stock_data_long_format["Target"] = stock_data_long_format.groupby('Ticker')['Close'].pct_change(3).shift(-3) 

create_features(stock_data_long_format)

stock_data_long_format[feature_columns] = stock_data_long_format.groupby('Ticker')[feature_columns].shift(1) # prevent data leakage from using same-day data
stock_data_long_format.dropna()

backtesting_data = stock_data_long_format


C:\Users\maria\AppData\Local\Temp\ipykernel_25116\1650103216.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  20 of 20 completed
C:\Users\maria\AppData\Local\Temp\ipykernel_25116\1650103216.py:13: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stock_data.stack(level=1).reset_index()
C:\Users\maria\AppData\Local\Temp\ipykernel_25116\1650103216.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  macro_data = yf.download(macro_tickers, start=start_date, end=end_date)['Close']
[*********************100%***********************]  3 of 3 completed
C:\Users\maria\AppData\Local\Temp\ipykernel_25116\1

KeyError: "Columns not found: 'Ticker_MSFT', 'Ticker_BA', 'Ticker_PFE', 'Ticker_JNJ', 'Ticker_VTI', 'Ticker_SPY', 'Ticker_AMZN', 'Ticker_NEE', 'Ticker_T', 'Ticker_GOOG', 'Ticker_DIS', 'Ticker_KO', 'Ticker_AAPL', 'Ticker_QQQ', 'Ticker_XOM', 'Ticker_BAC', 'Ticker_BHP', 'Ticker_IWM', 'Ticker_NVDA', 'Ticker_JPM'"

For now i'm gonna ignore the tickers
We'll see if it actually helps with prediciton later on

In [2]:
# make predictions:
X_backtest = backtesting_data[feature_columns]
y_backtest = backtesting_data["Target"]
backtest_pred = model.predict(X_backtest)

NameError: name 'backtesting_data' is not defined

In [29]:
backtesting_data["Prediction"] = backtest_pred

In [1]:
backtesting_data[abs(backtesting_data['Prediction']) > 0.05] # keep only strong signals

NameError: name 'backtesting_data' is not defined

In [32]:
def compute_daily_returns_per_stock(df, top_pct=0.2, bottom_pct=0.2):
    """
    Compute daily returns per stock using regression predictions.
    
    df must have ['Date', 'Ticker', 'Prediction', 'Target']
    """
    df = df.copy()
    df['StrategyReturn'] = 0.0

    for date, group in df.groupby('Date'):
        group = group.copy()
        n_stocks = len(group)
        n_top = max(1, int(top_pct * n_stocks))
        n_bottom = max(1, int(bottom_pct * n_stocks))

        # Long top predicted stocks, short bottom predicted stocks
        top_stocks = group.nlargest(n_top, 'Prediction')['Ticker']
        bottom_stocks = group.nsmallest(n_bottom, 'Prediction')['Ticker']

        # Use actual Target for realized return
        df.loc[(df['Date'] == date) & (df['Ticker'].isin(top_stocks)), 'StrategyReturn'] = \
            group.loc[group['Ticker'].isin(top_stocks), 'Target']
        df.loc[(df['Date'] == date) & (df['Ticker'].isin(bottom_stocks)), 'StrategyReturn'] = \
            -group.loc[group['Ticker'].isin(bottom_stocks), 'Target']

        # All other stocks remain 0
    return df[['Date', 'Ticker', 'Prediction', 'Target', 'StrategyReturn']]



def stock_performance_strategy(df):
    """
    Compute per-stock metrics using StrategyReturn.
    """
    stock_perf = []

    for ticker, group in df.groupby('Ticker'):
        trades_df = group[group['StrategyReturn'] != 0]
        trades = len(trades_df)
        wins = (trades_df['StrategyReturn'] > 0).sum()
        avg_return = trades_df['StrategyReturn'].mean() if trades > 0 else 0.0
        sharpe = avg_return / trades_df['StrategyReturn'].std() if trades > 1 and trades_df['StrategyReturn'].std() != 0 else avg_return

        stock_perf.append({
            'ticker': ticker,
            'trades': trades,
            'winRate': round(wins / trades, 2) if trades > 0 else 0.0,
            'avgReturn': round(avg_return, 3),
            'sharpe': round(sharpe, 2)
        })

    return stock_perf



In [36]:
daily_returns  = compute_daily_returns_per_stock(backtesting_data)

In [37]:
stock_performance_strategy(daily_returns)

[{'ticker': 'AAPL',
  'trades': 100,
  'winRate': np.float64(0.59),
  'avgReturn': np.float64(0.005),
  'sharpe': np.float64(0.15)},
 {'ticker': 'AMZN',
  'trades': 103,
  'winRate': np.float64(0.42),
  'avgReturn': np.float64(-0.006),
  'sharpe': np.float64(-0.26)},
 {'ticker': 'BA',
  'trades': 103,
  'winRate': np.float64(0.56),
  'avgReturn': np.float64(0.006),
  'sharpe': np.float64(0.18)},
 {'ticker': 'BAC',
  'trades': 83,
  'winRate': np.float64(0.53),
  'avgReturn': np.float64(-0.002),
  'sharpe': np.float64(-0.08)},
 {'ticker': 'BHP',
  'trades': 81,
  'winRate': np.float64(0.48),
  'avgReturn': np.float64(0.001),
  'sharpe': np.float64(0.05)},
 {'ticker': 'DIS',
  'trades': 59,
  'winRate': np.float64(0.53),
  'avgReturn': np.float64(0.004),
  'sharpe': np.float64(0.18)},
 {'ticker': 'GOOG',
  'trades': 101,
  'winRate': np.float64(0.54),
  'avgReturn': np.float64(0.0),
  'sharpe': np.float64(0.02)},
 {'ticker': 'IWM',
  'trades': 42,
  'winRate': np.float64(0.52),
  'avgRet

# get importnant info from backtest

# Evaluate model performance

In [ ]:
from scipy.stats import spearmanr

# How well do predictions correlate with actual returns?
ic, p_value = spearmanr(predictions, actual_returns)

print(f"Information Coefficient: {ic:.3f}")
# IC > 0.05 is decent, > 0.10 is very good in finance

In [ ]:
def evaluate_portfolio(predictions, actual_returns, n_quantiles=5):
    """
    Split predictions into quantiles, see if top quantile outperforms
    """
    df = pd.DataFrame({
        'pred': predictions,
        'actual': actual_returns
    })
    
    # Divide into quantiles
    df['quantile'] = pd.qcut(df['pred'], q=n_quantiles, labels=False)
    
    # Average return per quantile
    quantile_returns = df.groupby('quantile')['actual'].mean()
    
    print("Returns by Prediction Quantile:")
    print(quantile_returns)
    
    # Long-short return (top - bottom)
    long_short = quantile_returns.iloc[-1] - quantile_returns.iloc[0]
    print(f"\nLong-Short Return: {long_short:.4f}")
    
    return quantile_returns